#### Modified version

In [1]:
pip install streamlit-quill

   ---------------------------------------- 2.9/2.9 MB 19.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: C:\Users\85290\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
import fitz # pip install PyMuPDF
from datetime import date, timedelta
import html

TEMPLATE = "Weekly Newsletter Template v3.pdf"
doc      = fitz.open(TEMPLATE)

# ──────────────────── derive title & output
today    = date.today()
week_no  = int((today - timedelta(days=today.weekday())).strftime("%V"))
week_no = 17
week_tag    = f"Week {week_no}, {today.year}"
OUTPUT   = f"{today.year % 100}w{week_no:02d}.pdf"

# ──────────────────── pick up real font name (not alias!)
font_tag = None
fid, ftype, fsubtype, fname, tag, enc = doc[0].get_fonts()[0]
if "KaiseiTokumin" in fname:
	font_tag = tag         # e.g. 'F9'
	font_name = fname #Default font_name = "Helvetica"
assert font_tag, "Kaisei Tokumin not found!"

# ──────────────────── rectangles & payload
anchors = {
    "title":        fitz.Rect(12,  12, 588, 208),
    "events":       fitz.Rect(15, 252, 365, 428),
    "gratitude":    fitz.Rect(15, 448, 275, 528), #y0 decreased to accommodate bullet
    "productivity": fitz.Rect(15, 543, 275, 623), #y0 decreased to accommodate bullet
    "up_next":      fitz.Rect(15, 665, 340, 805),
    "facts":        fitz.Rect(290, 465, 585, 625),
    "img_rect":     fitz.Rect(375, 220, 585, 430),
    "weekly": 		fitz.Rect(359, 660, 464, 795),
}

payload = {
    "title": "Week of Math Education Revisit",
    "events": "<b>Math Education</b><ul><li><span style=\"color: rgb(240, 102, 102);\">Joined a 800-strong wts group</span> of math experts and students.</li><li>Spent ~45 hours over the week revamping my 2022 M2 reminder deck (MS Paint, 28 slides) to <span style=\"color: rgb(240, 102, 102);\">2025 ver (LaTeX beamer, 39 slides)</span>. Received 13k views on ig (exp. 10k).</li><li>Coordinated with a taskforce of another EMath student and two teachers on <span style=\"color: rgb(240, 102, 102);\">distributing/ upscaling resources (ongoing)</span>.</li></ul><h4>Personal</h4><ul><li><span style=\"color: rgb(240, 102, 102);\">Did Mensa</span> on Saturday. Comfortable entry if not careless.</li><li>Locked in <span style=\"color: rgb(240, 102, 102);\">HKPFS, other scholarships</span> and collaborations.</li><li>Devised auto <span style=\"color: rgb(240, 102, 102);\">text-to-html</span> <span style=\"color: rgb(240, 102, 102);\">converter</span> <a href = \"https://tsn-newsletter.streamlit.app/\">app</a></li></ul>",
    "gratitude": "<ul><li>Knowing more passionate, ambitious math edu fellows</li><li>Able to <span style=\"color: rgb(240, 102, 102);\">preserve/ enhance my edge</span> in math edu hobby</li><li>Recognition on <span style=\"color: rgb(240, 102, 102);\">previous work's impact</span> from student feedbacks/ teachers' praises of my free mock quality</li></ul><p><br></p>",
    "productivity": "<ul><li><strong style=\"color: rgb(32, 33, 34);\">Delayed sleep phase disorder</strong> (DSPD) is common in <span style=\"color: rgb(240, 102, 102);\">high-cognition, screen-heavy</span> lifestyles, but our <span style=\"color: rgb(240, 102, 102);\">circadian rhythms are plastic</span><span style=\"color: rgb(32, 33, 34);\"> (changeable easily). </span></li><li><span style=\"color: rgb(32, 33, 34);\">If allowed to </span><span style=\"color: rgb(240, 102, 102);\">follow own schedules</span><span style=\"color: rgb(32, 33, 34);\">, e.g. 6am to 3pm sleep, sleep quality and peak work efficiency could be preserved.</span></li></ul>",
    "up_next": "<b>Carried over from last week:</b><ul><li>Automate YT Insights via Py API library</li><li>Trade Strategy Expansion/ Testing on MDP</li><li>Document Lec4 CS234, study Lec 7-8 (Wrap up PPO)</li><li>Timetable matcher</li><li>Fina. data sentiment handler</li><li>Website v4 ecosystem/ wrapper</li><li>Upgrade newsletter with 5-10 insightful graphs per week</li></ul><p><br></p>",
    "facts": "<h4>Interaction btwn sleep deprivation and social func</h4><ul><li>Autism is linked to genetic mutations that affect melatonin regulation and circadian rhythm</li><li>Sleep disruption brings hyperactivity and easier distraction than those who sleep well</li><li>sleepiness predicts less social motivation (Axelsson et al., 2020), less social activity (Holding et al., 2020), and worsened social skills (induced autism?)</li></ul>",
    "weekly": "<p><a href=\"https://www.instagram.com/p/DIyh5SjzdAj/\" rel=\"noopener noreferrer\" target=\"_blank\">#99: Approximation</a> discusses philosophy of navigating life by \"just enough approximation w/o micromanaging\" and introducing \"psychology\" component alongside science.</p>",
    "img_rect": "25w17-image"
}

title = payload["title"]
payload["title"] = f"{week_tag}: {title}"
def make_html(src, is_title=False):
    if src.lstrip().startswith("<"):
        return src
    if is_title:
        return f'<h1 style="font-family:\'{font_name}\';margin:0">{html.escape(src)}</h1>'

    lines = src.splitlines()
    if all(l.startswith(("•", "➜", "-")) for l in lines if l.strip()):
        lis = "".join(f"<li>{html.escape(l.lstrip('•➜- '))}</li>"
                      for l in lines if l.strip())
        body = f"<ul>{lis}</ul>"
    else:
        body = "<br>".join(html.escape(l) for l in lines)

    return (f'<div style="font-family:\'{font_name}\';font-size:11pt;vertical-align:top;'
            f'line-height:13pt">{body}</div>')

# ──────────────────── draw on page 0
page = doc[0]
for key, rect in anchors.items():
    #if key == "img_rect":
    #    continue
    html_snip = make_html(payload[key], is_title=(key == "title"))
    try:
        # page.draw_rect(rect, color=(0, 0, 1), width=0.5) #Debug by displaying boundaries
        status = page.insert_htmlbox(rect, html_snip, scale_low=0.5, overlay=True)
        spare, scale = status
        if spare < 0:
            print(f"⚠️ '{key}' did not fully fit. scale={scale:.2f}, extra height needed: {-spare:.2f}pt")
    except OverflowError as e:
        print(f"🚨 OverflowError when inserting '{key}':", e)

# ───────── banner image + hyperlink
page.insert_image(anchors["img_rect"], filename=f"Published/{today.year % 100}w{week_no:02d}-image.png",
                  keep_proportion=True, overlay=True)
#page.add_link(anchors["img_rect"],
#              uri="https://github.com/your‑repo/newsletter",
#              kind=fitz.LINK_URI)

doc.save(OUTPUT, deflate=True, garbage=4)
print("✅  Saved", OUTPUT)

✅  Saved 25w17.pdf


#### 1a) Auto-mail to subscribers from a CSV

Caveat: smtp app password is required on google!

In [14]:
import smtplib
import ssl
import pandas as pd
from email.message import EmailMessage

email = "theskillfulnoob2002@gmail.com"
#pw = "the-SKILLFUL-2002"
app_pw = "wyxl fwfv vqpr udde"
def send_newsletter(csv_path, subject, plain_body, html_body, attachment_path=None, debug_email = None):
    if debug_email or not csv_path:
        bcc_list = [debug_email]
    else:
        df = pd.read_csv(csv_path)
        bcc_list = df['email'].tolist()
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = email
    msg['To'] = email  # visible To field (yourself or generic newsletter identity)
    msg['Bcc'] = ', '.join(bcc_list)
    msg.set_content(plain_body)
    msg.add_alternative(html_body, subtype='html')

    if attachment_path:
        with open(attachment_path, 'rb') as f:
            msg.add_attachment(f.read(), maintype='application', subtype='pdf',
                               filename=attachment_path)

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(email, app_pw)
        smtp.send_message(msg)

#### 1b) HTML Format Editor

In [20]:
html_content = f"""
<html>
  <body style="font-family:Arial, sans-serif; margin:0; padding:0;">
    <div style="background-color:#E0F7FA; padding:24px;">
      <div style="margin-top:20px;">
		<div style="background-color:#FFFACD; padding:16px; border-radius:8px;">
			<h2 style="color:#4CAF50; margin-top:0;">Your Weekly Digest</h2>
			<p>Hi there! ☀️</p>
			<p>Here's another week of your newsletter - attached as a PDF!</p>
			<br><br>
			<p>Have a good Labour Day holiday ahead :)</p>
		</div>
  
        <p style="font-style:italic; color: #3994cc;">Stay inspired and informed.
			<br>Feel free to <b>give me feedback</b> from time to time - love to hear from you too! ❤️
			<br>- Andrew
		</p>
      </div>
    </div>

    <div style="background-color:#F0F0F0; padding:16px; font-size:0.9em; color:#555; text-align:center;">
      You received this email because you're subscribed to Andrew's Weekly Newsletter.
      <br>
      You can <a href="mailto:{email}">Unsubscribe</a> by emailing me.
    </div>
  </body>
</html>
"""

## TOGGLE SWITCH

In [ ]:
subject = f"🌱 2025 W{week_no} Newsletter: {title}"
attachment = f"Published/25w{week_no}.pdf"
csv_path = "recipients.csv"
plain_text = f"Your Week {week_no} newsletter is attached. If you cannot view the HTML version, please check the PDF."
send_newsletter(#csv_path=csv_path, #uncomment this for production
                csv_path = None, #comment this for production
                subject=subject, 
                plain_body=plain_text, 
                html_body=html_content, 
                attachment_path=attachment,
                debug_email = email, #comment this for production
                )

In [23]:
msg["bcc"]

NameError: name 'msg' is not defined

#### 1c) Add and Drop Subscribers

In [13]:
import pandas as pd
from datetime import date
csv_path = "recipients.csv"
def modify_subscription(email, csv_path, action='add'):
    df = pd.read_csv(csv_path)
    if action == 'add':
        if email not in df['email'].values:
            new_entry = pd.DataFrame([[email, date.today().isoformat()]], columns=['email', 'subscribed_date'])
            df = pd.concat([df, new_entry], ignore_index=True)
            print(f"Added {email} to the list!")
        else:
            print(f"{email} already exists on the list!")
    elif action == 'drop':
        df = df[df['email'] != email]
    df.to_csv(csv_path, index=False)

for add in ['chanberenice3@gmail.com', 'kiandra0109@gmail.com']:
    modify_subscription(add, csv_path, 'add')

Added chanberenice3@gmail.com to the list!
Added kiandra0109@gmail.com to the list!
